In [1]:
import sys
import json
import os
import re
import shutil
import subprocess
import random
import PIL.Image
import cv2
import numpy as np
from collections import OrderedDict
import copy
# import xmltodict
import sys
import math
import time
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
# import cStringIO as StringIO
import csv
import pandas
import xlrd
# import dicom
# from scipy.misc import imresize
from skimage.transform import rescale, resize
import os
import pandas as pd
import scipy.ndimage.interpolation
import math

In [2]:
# Load lesion data
# Root directory of the project
ROOT_DIR = os.getcwd()
print(ROOT_DIR)
# if ROOT_DIR.endswith("samples/nucleus"):
if ROOT_DIR.endswith("mammography"):
    # Go up two levels to the repo root
    ROOT_DIR = os.path.dirname(ROOT_DIR)
    print(ROOT_DIR)

DATASET_DIR = os.path.join(ROOT_DIR, "datasets/mammo")
IMAGES_DIR = "/home/chevy/Mammo/"
file_name = "images_metadata.xlsx"
sheet_name="LesionMarkAttributesWithImageDim"
file_path = os.path.join(DATASET_DIR, file_name)
tsv_annotations = pd.read_excel(file_path, sheet_name=sheet_name)



/home/chevy/Desktop/Github/Mammo_MaskRCNN/mammography
/home/chevy/Desktop/Github/Mammo_MaskRCNN


In [10]:
# Initialise
xml_annotations = {'type': 'instances', 
                   'annotations': [], 
                   'images': [], 
                   'categories': [{'id': 1, 'name': 'MALIGNANT'}, {'id': 2, 'name': 'BENIGN'}, 
                                  {'id': 3, 'name': 'NORMAL'}]
                  }
# IMAGE_MAX_DIM = 3328
IMAGE_MAX_DIM = 1024
ImageSOPIUIDtoID = {}
image_id = 1
anno_id = 1
annos = []

## Ensure directory exists if not created directory
## TODO: Move above so this won't have to be rerun every iteration
aug_img_path = os.path.join(DATASET_DIR, "augmented_images_test")
aug_anno_img_path = os.path.join(DATASET_DIR, "augmented_images_test")

if not os.path.exists(aug_img_path):
    os.mkdir(aug_img_path)
if not os.path.exists(aug_anno_img_path):
    os.mkdir(aug_anno_img_path)

In [ ]:
DATA_SET = 1

start = time.time()
for i in range(0, len(tsv_annotations)):
# for i in range(3508, 3515):
    row = tsv_annotations.loc[i, :]
    print(i)

    if row['ImageSOPIUID'] == '1.2.840.113619.2.227.2079161146212.2295131007153007.103':
        continue
    if row['Status'] == 'B1':
        continue
    assert row['Status'] in ['Malignant', 'Benign', 'B2', 'B3', 'B4', 'B5']
    
    # Check box dimensions
    box_width = row['Width']
    box_height = row['Height']
    if box_width <= 0 or box_height <= 0:
        print('box width/heigth <= 0')
        continue 
    image = cv2.imread(IMAGES_DIR + row['ImageSOPIUID'] + '.png')
    assert box_width == row['X2'] - row['X1']
    if not row['X2'] <= image.shape[1]:
        print(row['X2'], image.shape[1])
        print('X2 more than width')
        print(row['ImageSOPIUID'])
        row['X2'] = image.shape[1]
    assert box_height == row['Y2'] - row['Y1']
    if not row['Y2'] <= image.shape[0]:
        print(row['Y2'], image.shape[0] )
        print('Y2 more than Height')
        print(row['ImageSOPIUID'])
        row['Y2'] = image.shape[0]
    
    # Rescale boxes
    scaling_factor = float(IMAGE_MAX_DIM) / float(np.max(image.shape))
    print("Scaling factor:", scaling_factor)   
    image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
#     skimage.transform.resize
    box_width = int(scaling_factor * box_width)
    box_height = int(scaling_factor * box_height)
    row['X1'] = int(scaling_factor * row['X1'])
    row['Y1'] = int(scaling_factor * row['Y1'])
    row['X2'] = int(scaling_factor * row['X2'])
    row['Y2'] = int(scaling_factor * row['Y2'])

    # Make Augmentations
    image_before = image
    for rot in range(0, 4):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
#     for rot in range(0, 1):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
        image = copy.deepcopy(image_before)
        image = scipy.ndimage.interpolation.rotate(image, angle=90*rot)

        im_name = row['ImageSOPIUID'] + '_rot' + str(rot)
        if im_name not in ImageSOPIUIDtoID:
            ImageSOPIUIDtoID[im_name] = image_id
            image_id += 1
            xml_annotations['images'] += [
                {'file_name': im_name + '.png',
                 'height': image.shape[0],
                 'width': image.shape[1],
                 'id': ImageSOPIUIDtoID[im_name]
                }
            ]

        if rot:  #  rot == 1 to rot == 3
            box_width, box_height = box_height, box_width
        
        x = (row['X1'] + row['X2']) / 2
        y = (row['Y1'] + row['Y2']) / 2
        if rot == 0:
            xx, yy = x, y
        if rot == 1:  # 90 deg anticlockwise
            xx = y
            yy = image_before.shape[1] - x
        if rot == 2:
            xx = image_before.shape[1] - x
            yy = image_before.shape[0] - y
        if rot == 3:
            xx = image_before.shape[0] - y
            yy = x
        x, y = xx, yy
        
        anno = {}
#         anno['og_bbox'] = [x - box_width / 2, y - box_height / 2, box_width, box_height]
        x1 = x - box_width / 2
        y1 = y - box_height / 2
        x2 = x1 + box_width
        y2 = y1 + box_height
        anno['bbox'] = [x1, y1, x2, y2]
        anno['area'] = box_width * box_height
        anno['iscrowd'] = 0
        anno['image_id'] = ImageSOPIUIDtoID[im_name]
        anno['category_id'] = 1 if row['Status'] in ['Malignant', 'B4', 'B5'] else 2
        anno['id'] = anno_id
        anno_id += 1
        xml_annotations['annotations'] += [anno]

       
#         image_path = os.path.join(aug_img_path, im_name + '.png')
#         cv2.imwrite(image_path, image)
#         pt1 = (int(anno['og_bbox'][0]), int(anno['og_bbox'][1]))
#         pt2 = (int(anno['og_bbox'][0] + anno['og_bbox'][2]), int(anno['og_bbox'][1] + anno['og_bbox'][3]))
#         print("rot", rot)
#         print("pt1:" , pt1)
#         print("pt2:", pt2)
#         cv2.rectangle(image, pt1, pt2 , 
#                      (255, 255 * anno['category_id'] == 2, 255 * anno['category_id'] == 1),
#                      4)
#         anno_img_path = os.path.join(aug_anno_img_path, im_name + '.png')
#         cv2.imwrite(anno_img_path, image)

elapsed_time = time.time() - start
print("Total time taken:", elapsed_time)

with open(DATASET_DIR + '_4x_updated.json', 'w') as fp:
    json.dump(xml_annotations, fp, indent=4)
    

0
Scaling factor: 0.25


/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:46

1
Scaling factor: 0.25
2
Scaling factor: 0.25
3
Scaling factor: 0.25
4
Scaling factor: 0.3076923076923077
5
Scaling factor: 0.3076923076923077
6
Scaling factor: 0.25
7
Scaling factor: 0.3076923076923077
8
Scaling factor: 0.3076923076923077
9
Scaling factor: 0.3076923076923077
10
Scaling factor: 0.3076923076923077
11
Scaling factor: 0.3076923076923077
12
Scaling factor: 0.33442194644023515
13
Scaling factor: 0.33442194644023515
14
Scaling factor: 0.25
15
Scaling factor: 0.25
16
Scaling factor: 0.3076923076923077
17
Scaling factor: 0.3076923076923077
18
Scaling factor: 0.3076923076923077
19
Scaling factor: 0.3076923076923077
20
Scaling factor: 0.3076923076923077
21
Scaling factor: 0.3076923076923077
22
Scaling factor: 0.3076923076923077
23
Scaling factor: 0.3076923076923077
24
Scaling factor: 0.44638186573670446
25
Scaling factor: 0.44638186573670446
26
Scaling factor: 0.25
27
Scaling factor: 0.25
28
Scaling factor: 0.3076923076923077
29
Scaling factor: 0.3076923076923077
30
Scaling fact

In [6]:
DATA_SET = 1

start = time.time()
for i in range(0, len(tsv_annotations)):
# for i in range(3508, 3515):
    row = tsv_annotations.loc[i, :]
    print(i)

    if row['ImageSOPIUID'] == '1.2.840.113619.2.227.2079161146212.2295131007153007.103':
        continue
    if row['Status'] == 'B1':
        continue
    assert row['Status'] in ['Malignant', 'Benign', 'B2', 'B3', 'B4', 'B5']
    
    # Check box dimensions
    box_width = row['Width']
    box_height = row['Height']
    if box_width <= 0 or box_height <= 0:
        print('box width/heigth <= 0')
        continue 
    image = cv2.imread(IMAGES_DIR + row['ImageSOPIUID'] + '.png')
    assert box_width == row['X2'] - row['X1']
    if not row['X2'] <= image.shape[1]:
        print(row['X2'], image.shape[1])
        print('X2 more than width')
        print(row['ImageSOPIUID'])
        row['X2'] = image.shape[1]
    assert box_height == row['Y2'] - row['Y1']
    if not row['Y2'] <= image.shape[0]:
        print(row['Y2'], image.shape[0] )
        print('Y2 more than Height')
        print(row['ImageSOPIUID'])
        row['Y2'] = image.shape[0]
    
    # Rescale boxes
    scaling_factor = float(IMAGE_MAX_DIM) / float(np.max(image.shape))
    print("Scaling factor:", scaling_factor)   
    image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
#     skimage.transform.resize
    box_width = int(scaling_factor * box_width)
    box_height = int(scaling_factor * box_height)
    row['X1'] = int(scaling_factor * row['X1'])
    row['Y1'] = int(scaling_factor * row['Y1'])
    row['X2'] = int(scaling_factor * row['X2'])
    row['Y2'] = int(scaling_factor * row['Y2'])

    # Make Augmentations
    image_before = image
    for rot in range(0, 4):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
#     for rot in range(0, 1):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
        image = copy.deepcopy(image_before)
        image = scipy.ndimage.interpolation.rotate(image, angle=90*rot)

        im_name = row['ImageSOPIUID'] + '_rot' + str(rot)
        if im_name not in ImageSOPIUIDtoID:
            ImageSOPIUIDtoID[im_name] = image_id
            image_id += 1
            xml_annotations['images'] += [
                {'file_name': im_name + '.png',
                 'height': image.shape[0],
                 'width': image.shape[1],
                 'id': ImageSOPIUIDtoID[im_name]
                }
            ]

        if rot:  #  rot == 1 to rot == 3
            box_width, box_height = box_height, box_width
        
        x = (row['X1'] + row['X2']) / 2
        y = (row['Y1'] + row['Y2']) / 2
        if rot == 0:
            xx, yy = x, y
        if rot == 1:  # 90 deg anticlockwise
            xx = y
            yy = image_before.shape[1] - x
        if rot == 2:
            xx = image_before.shape[1] - x
            yy = image_before.shape[0] - y
        if rot == 3:
            xx = image_before.shape[0] - y
            yy = x
        x, y = xx, yy
        
        anno = {}
#         anno['og_bbox'] = [x - box_width / 2, y - box_height / 2, box_width, box_height]
        x1 = x - box_width / 2
        y1 = y - box_height / 2
        x2 = x1 + box_width
        y2 = y1 + box_height
        anno['bbox'] = [x1, y1, x2, y2]
        anno['area'] = box_width * box_height
        anno['iscrowd'] = 0
        anno['image_id'] = ImageSOPIUIDtoID[im_name]
        anno['category_id'] = 1 if row['Status'] in ['Malignant', 'B4', 'B5'] else 2
        anno['id'] = anno_id
        anno_id += 1
        xml_annotations['annotations'] += [anno]

       
#         image_path = os.path.join(aug_img_path, im_name + '.png')
#         cv2.imwrite(image_path, image)
#         pt1 = (int(anno['og_bbox'][0]), int(anno['og_bbox'][1]))
#         pt2 = (int(anno['og_bbox'][0] + anno['og_bbox'][2]), int(anno['og_bbox'][1] + anno['og_bbox'][3]))
#         print("rot", rot)
#         print("pt1:" , pt1)
#         print("pt2:", pt2)
#         cv2.rectangle(image, pt1, pt2 , 
#                      (255, 255 * anno['category_id'] == 2, 255 * anno['category_id'] == 1),
#                      4)
#         anno_img_path = os.path.join(aug_anno_img_path, im_name + '.png')
#         cv2.imwrite(anno_img_path, image)

elapsed_time = time.time() - start
print("Total time taken:", elapsed_time)

with open(DATASET_DIR + '_4x.json', 'w') as fp:
    json.dump(xml_annotations, fp, indent=4)
    

0
Scaling factor: 0.25
1


/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:46

Scaling factor: 0.25
2
Scaling factor: 0.25
3
Scaling factor: 0.25
4
Scaling factor: 0.3076923076923077
5
Scaling factor: 0.3076923076923077
6
Scaling factor: 0.25
7
Scaling factor: 0.3076923076923077
8
Scaling factor: 0.3076923076923077
9
Scaling factor: 0.3076923076923077
10
Scaling factor: 0.3076923076923077
11
Scaling factor: 0.3076923076923077
12
Scaling factor: 0.33442194644023515
13
Scaling factor: 0.33442194644023515
14
Scaling factor: 0.25
15
Scaling factor: 0.25
16
Scaling factor: 0.3076923076923077
17
Scaling factor: 0.3076923076923077
18
Scaling factor: 0.3076923076923077
19
Scaling factor: 0.3076923076923077
20
Scaling factor: 0.3076923076923077
21
Scaling factor: 0.3076923076923077
22
Scaling factor: 0.3076923076923077
23
Scaling factor: 0.3076923076923077
24
Scaling factor: 0.44638186573670446
25
Scaling factor: 0.44638186573670446
26
Scaling factor: 0.25
27
Scaling factor: 0.25
28
Scaling factor: 0.3076923076923077
29
Scaling factor: 0.3076923076923077
30
Scaling factor

Scaling factor: 0.25
235
Scaling factor: 0.25
236
Scaling factor: 0.3076923076923077
237
Scaling factor: 0.3076923076923077
238
Scaling factor: 0.25
239
Scaling factor: 0.25
240
Scaling factor: 0.25
241
Scaling factor: 0.25
242
Scaling factor: 0.3076923076923077
243
Scaling factor: 0.3076923076923077
244
Scaling factor: 0.3076923076923077
245
Scaling factor: 0.3076923076923077
246
Scaling factor: 0.25
247
Scaling factor: 0.25
248
Scaling factor: 0.25
249
Scaling factor: 0.25
250
Scaling factor: 0.3076923076923077
251
Scaling factor: 0.3076923076923077
252
Scaling factor: 0.25
253
Scaling factor: 0.25
254
Scaling factor: 0.3076923076923077
255
Scaling factor: 0.3076923076923077
256
Scaling factor: 0.3076923076923077
257
Scaling factor: 0.3076923076923077
258
Scaling factor: 0.25
259
Scaling factor: 0.25
260
Scaling factor: 0.25
261
Scaling factor: 0.25
262
Scaling factor: 0.25
263
Scaling factor: 0.25
264
Scaling factor: 0.3076923076923077
265
Scaling factor: 0.3076923076923077
266
Scal

Scaling factor: 0.3076923076923077
491
Scaling factor: 0.3076923076923077
492
Scaling factor: 0.3076923076923077
493
Scaling factor: 0.3076923076923077
494
Scaling factor: 0.3076923076923077
495
Scaling factor: 0.3076923076923077
496
Scaling factor: 0.3076923076923077
497
Scaling factor: 0.3076923076923077
498
Scaling factor: 0.3076923076923077
499
Scaling factor: 0.3076923076923077
500
Scaling factor: 0.3076923076923077
501
Scaling factor: 0.3076923076923077
502
Scaling factor: 0.3076923076923077
503
Scaling factor: 0.3076923076923077
504
Scaling factor: 0.3076923076923077
505
Scaling factor: 0.25
506
Scaling factor: 0.3076923076923077
507
Scaling factor: 0.3076923076923077
508
Scaling factor: 0.3076923076923077
509
Scaling factor: 0.3076923076923077
510
Scaling factor: 0.3076923076923077
511
Scaling factor: 0.3076923076923077
512
Scaling factor: 0.3076923076923077
513
Scaling factor: 0.3076923076923077
514
Scaling factor: 0.3076923076923077
515
Scaling factor: 0.25
516
Scaling factor

...

Scaling factor: 0.2507345739471107
6560
Scaling factor: 0.2507345739471107
6561
Scaling factor: 0.2507345739471107
6562
Scaling factor: 0.25
6563
Scaling factor: 0.25
6564
Scaling factor: 0.3076923076923077
6565
Scaling factor: 0.2507345739471107
6566
Scaling factor: 0.2507345739471107
6567
Scaling factor: 0.25
6568
Scaling factor: 0.25
6569
Scaling factor: 0.25
6570
Scaling factor: 0.25
6571
Scaling factor: 0.3076923076923077
6572
Scaling factor: 0.3076923076923077
6573
Scaling factor: 0.3076923076923077
6574
Scaling factor: 0.3076923076923077
6575
Scaling factor: 0.25
6576
Scaling factor: 0.25
6577
Scaling factor: 0.2507345739471107
6578
Scaling factor: 0.2507345739471107
6579
Scaling factor: 0.3076923076923077
6580
Scaling factor: 0.3076923076923077
6581
Scaling factor: 0.25
6582
Scaling factor: 0.25
6583
Scaling factor: 0.3076923076923077
6584
Scaling factor: 0.3076923076923077
6585
Scaling factor: 0.3076923076923077
6586
Scaling factor: 0.25
6587
Scaling factor: 0.25
6588
Scaling 

Scaling factor: 0.2507345739471107
6799
Scaling factor: 0.2507345739471107
6800
Scaling factor: 0.25
6801
Scaling factor: 0.25
6802
Scaling factor: 0.25
6803
Scaling factor: 0.25
6804
Scaling factor: 0.25
6805
Scaling factor: 0.25
6806
Scaling factor: 0.3076923076923077
6807
Scaling factor: 0.3076923076923077
6808
Scaling factor: 0.3076923076923077
6809
Scaling factor: 0.3076923076923077
6810
Scaling factor: 0.3076923076923077
6811
Scaling factor: 0.3076923076923077
6812
Scaling factor: 0.25
6813
Scaling factor: 0.25
6814
Scaling factor: 0.25
6815
Scaling factor: 0.25
6816
Scaling factor: 0.33442194644023515
6817
Scaling factor: 0.33442194644023515
6818
Scaling factor: 0.25
6819
Scaling factor: 0.25
6820
Scaling factor: 0.25
6821
Scaling factor: 0.3076923076923077
6822
Scaling factor: 0.3076923076923077
6823
Scaling factor: 0.25
6824
Scaling factor: 0.25
6825
Scaling factor: 0.25
6826
Scaling factor: 0.25
6827
Scaling factor: 0.3076923076923077
6828
Scaling factor: 0.3076923076923077
6

7036
Scaling factor: 0.19122315592903827
7037
Scaling factor: 0.19122315592903827
7038
Scaling factor: 0.19122315592903827
7039
Scaling factor: 0.19122315592903827
7040
Scaling factor: 0.19122315592903827
7041
Scaling factor: 0.3076923076923077
7042
Scaling factor: 0.3076923076923077
7043
Scaling factor: 0.19122315592903827
7044
Scaling factor: 0.19122315592903827
7045
Scaling factor: 0.19122315592903827
7046
Scaling factor: 0.19122315592903827
7047
Scaling factor: 0.19122315592903827
7048
Scaling factor: 0.19122315592903827
7049
Scaling factor: 0.19122315592903827
7050
Scaling factor: 0.19122315592903827
7051
Scaling factor: 0.19122315592903827
7052
Scaling factor: 0.19122315592903827
7053
Scaling factor: 0.19122315592903827
7054
Scaling factor: 0.19122315592903827
7055
Scaling factor: 0.19122315592903827
7056
Scaling factor: 0.19122315592903827
7057
Scaling factor: 0.19122315592903827
7058
Scaling factor: 0.19122315592903827
7059
Scaling factor: 0.19122315592903827
7060
Scaling facto

Scaling factor: 0.25
7278
Scaling factor: 0.25
7279
Scaling factor: 0.25
7280
Scaling factor: 0.25
7281
Scaling factor: 0.3076923076923077
7282
Scaling factor: 0.3076923076923077
7283
Scaling factor: 0.25
7284
Scaling factor: 0.25
7285
Scaling factor: 0.3076923076923077
7286
Scaling factor: 0.3076923076923077
7287
Scaling factor: 0.25
7288
Scaling factor: 0.25
7289
Scaling factor: 0.3076923076923077
7290
Scaling factor: 0.3076923076923077
7291
Scaling factor: 0.3076923076923077
7292
Scaling factor: 0.3076923076923077
7293
Scaling factor: 0.3076923076923077
7294
Scaling factor: 0.3076923076923077
7295
Scaling factor: 0.25
7296
Scaling factor: 0.25
7297
Scaling factor: 0.3076923076923077
7298
Scaling factor: 0.3076923076923077
7299
Scaling factor: 0.25
7300
Scaling factor: 0.25
7301
Scaling factor: 0.25
7302
Scaling factor: 0.25
7303
Scaling factor: 0.25
7304
Scaling factor: 0.25
7305
Scaling factor: 0.25
7306
Scaling factor: 0.25
7307
Scaling factor: 0.3076923076923077
7308
Scaling fact

7537
Scaling factor: 0.25
7538
Scaling factor: 0.25
7539
Scaling factor: 0.25
7540
Scaling factor: 0.25
7541
Scaling factor: 0.25
7542
Scaling factor: 0.25
7543
Scaling factor: 0.25
7544
Scaling factor: 0.25
7545
Scaling factor: 0.3076923076923077
7546
Scaling factor: 0.3076923076923077
7547
Scaling factor: 0.3076923076923077
7548
Scaling factor: 0.3076923076923077
7549
Scaling factor: 0.25
7550
Scaling factor: 0.25
7551
Scaling factor: 0.25
7552
Scaling factor: 0.25
7553
Scaling factor: 0.3076923076923077
7554
Scaling factor: 0.3076923076923077
7555
Scaling factor: 0.3076923076923077
7556
Scaling factor: 0.3076923076923077
7557
Scaling factor: 0.25
7558
Scaling factor: 0.25
7559
Scaling factor: 0.3076923076923077
7560
Scaling factor: 0.3076923076923077
7561
Scaling factor: 0.25
7562
Scaling factor: 0.25
7563
Scaling factor: 0.25
7564
Scaling factor: 0.25
7565
Scaling factor: 0.3076923076923077
7566
Scaling factor: 0.3076923076923077
7567
Scaling factor: 0.25
7568
Scaling factor: 0.25


In [7]:
# Initialise
xml_annotations = {'type': 'instances', 
                   'annotations': [], 
                   'images': [], 
                   'categories': [{'id': 1, 'name': 'MALIGNANT'}, {'id': 2, 'name': 'BENIGN'}, 
                                  {'id': 3, 'name': 'NORMAL'}]
                  }
# IMAGE_MAX_DIM = 3328
IMAGE_MAX_DIM = 1024
ImageSOPIUIDtoID = {}
image_id = 1
anno_id = 1
annos = []

## Ensure directory exists if not created directory
## TODO: Move above so this won't have to be rerun every iteration
aug_img_path = os.path.join(DATASET_DIR, "augmented_images_test")
aug_anno_img_path = os.path.join(DATASET_DIR, "augmented_images_test")

if not os.path.exists(aug_img_path):
    os.mkdir(aug_img_path)
if not os.path.exists(aug_anno_img_path):
    os.mkdir(aug_anno_img_path)

In [8]:
DATA_SET = 1

start = time.time()
for i in range(0, len(tsv_annotations)):
# for i in range(3508, 3515):
    row = tsv_annotations.loc[i, :]
    print(i)
#     print(row['batch'])
    
    # Omit some images, check status
#     if row['batch'] != 1:
#         continue
    # image of some metal part, no breast here
    if row['ImageSOPIUID'] == '1.2.840.113619.2.227.2079161146212.2295131007153007.103':
        continue
    if row['Status'] == 'B1':
        continue
    assert row['Status'] in ['Malignant', 'Benign', 'B2', 'B3', 'B4', 'B5']
    
    # Check box dimensions
    box_width = row['Width']
    box_height = row['Height']
    if box_width <= 0 or box_height <= 0:
        print('box width/heigth <= 0')
        continue 
#     image = cv2.imread(ROOT_DIR + 'surrey_png_fullres/' +  row['ImageSOPIUID'] + '.png')
    image = cv2.imread(IMAGES_DIR + row['ImageSOPIUID'] + '.png')
    assert box_width == row['X2'] - row['X1']
    if not row['X2'] <= image.shape[1]:
        print(row['X2'], image.shape[1])
        print('X2 more than width')
        print(row['ImageSOPIUID'])
        row['X2'] = image.shape[1]
    assert box_height == row['Y2'] - row['Y1']
    if not row['Y2'] <= image.shape[0]:
        print(row['Y2'], image.shape[0] )
        print('Y2 more than Height')
        print(row['ImageSOPIUID'])
        row['Y2'] = image.shape[0]
    
    # Rescale boxes
    scaling_factor = float(IMAGE_MAX_DIM) / float(np.max(image.shape))
    print("Scaling factor:", scaling_factor)   
#     image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
#     skimage.transform.resize
    box_width = int(scaling_factor * box_width)
    box_height = int(scaling_factor * box_height)
    row['X1'] = int(scaling_factor * row['X1'])
    row['Y1'] = int(scaling_factor * row['Y1'])
    row['X2'] = int(scaling_factor * row['X2'])
    row['Y2'] = int(scaling_factor * row['Y2'])

    # Make Augmentations
    image_before = image
#     for rot in range(0, 4):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
    for rot in range(0, 1):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
        image = copy.deepcopy(image_before)
        image = scipy.ndimage.interpolation.rotate(image, angle=90*rot)

        im_name = row['ImageSOPIUID'] + '_rot' + str(rot)
        if im_name not in ImageSOPIUIDtoID:
            ImageSOPIUIDtoID[im_name] = image_id
            image_id += 1
            xml_annotations['images'] += [
                {'file_name': im_name + '.png',
                 'height': image.shape[0],
                 'width': image.shape[1],
                 'id': ImageSOPIUIDtoID[im_name]
                }
            ]

        if rot:  #  rot == 1 to rot == 3
            box_width, box_height = box_height, box_width
        
        x = (row['X1'] + row['X2']) / 2
        y = (row['Y1'] + row['Y2']) / 2
        if rot == 0:
            xx, yy = x, y
        if rot == 1:  # 90 deg anticlockwise
            xx = y
            yy = image_before.shape[1] - x
        if rot == 2:
            xx = image_before.shape[1] - x
            yy = image_before.shape[0] - y
        if rot == 3:
            xx = image_before.shape[0] - y
            yy = x
        x, y = xx, yy
        
        anno = {}
#         anno['og_bbox'] = [x - box_width / 2, y - box_height / 2, box_width, box_height]
        x1 = x - box_width / 2
        y1 = y - box_height / 2
        x2 = x1 + box_width
        y2 = y1 + box_height
        anno['bbox'] = [x1, y1, x2, y2]
        anno['area'] = box_width * box_height
        anno['iscrowd'] = 0
        anno['image_id'] = ImageSOPIUIDtoID[im_name]
        anno['category_id'] = 1 if row['Status'] in ['Malignant', 'B4', 'B5'] else 2
        anno['id'] = anno_id
#         anno['file_name']: im_name + '.png'
#         anno['height']: image.shape[0]
#         anno['width']: image.shape[1]
#         anno['id']: ImageSOPIUIDtoID[im_name]
        anno_id += 1
        xml_annotations['annotations'] += [anno]

       
#         image_path = os.path.join(aug_img_path, im_name + '.png')
#         cv2.imwrite(image_path, image)
#         pt1 = (int(anno['og_bbox'][0]), int(anno['og_bbox'][1]))
#         pt2 = (int(anno['og_bbox'][0] + anno['og_bbox'][2]), int(anno['og_bbox'][1] + anno['og_bbox'][3]))
#         print("rot", rot)
#         print("pt1:" , pt1)
#         print("pt2:", pt2)
#         cv2.rectangle(image, pt1, pt2 , 
#                      (255, 255 * anno['category_id'] == 2, 255 * anno['category_id'] == 1),
#                      4)
#         anno_img_path = os.path.join(aug_anno_img_path, im_name + '.png')
#         cv2.imwrite(anno_img_path, image)

elapsed_time = time.time() - start
print("Total time taken:", elapsed_time)

with open(DATASET_DIR + '_normal.json', 'w') as fp:
    json.dump(xml_annotations, fp, indent=4)
    

0
Scaling factor: 0.25
1


/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chevy/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:52

Scaling factor: 0.25
2
Scaling factor: 0.25
3
Scaling factor: 0.25
4
Scaling factor: 0.3076923076923077
5
Scaling factor: 0.3076923076923077
6
Scaling factor: 0.25
7
Scaling factor: 0.3076923076923077
8
Scaling factor: 0.3076923076923077
9
Scaling factor: 0.3076923076923077
10
Scaling factor: 0.3076923076923077
11
Scaling factor: 0.3076923076923077
12
Scaling factor: 0.33442194644023515
13
Scaling factor: 0.33442194644023515
14
Scaling factor: 0.25
15
Scaling factor: 0.25
16
Scaling factor: 0.3076923076923077
17
Scaling factor: 0.3076923076923077
18
Scaling factor: 0.3076923076923077
19
Scaling factor: 0.3076923076923077
20
Scaling factor: 0.3076923076923077
21
Scaling factor: 0.3076923076923077
22
Scaling factor: 0.3076923076923077
23
Scaling factor: 0.3076923076923077
24
Scaling factor: 0.44638186573670446
25
Scaling factor: 0.44638186573670446
26
Scaling factor: 0.25
27
Scaling factor: 0.25
28
Scaling factor: 0.3076923076923077
29
Scaling factor: 0.3076923076923077
30
Scaling factor

Scaling factor: 0.25
235
Scaling factor: 0.25
236
Scaling factor: 0.3076923076923077
237
Scaling factor: 0.3076923076923077
238
Scaling factor: 0.25
239
Scaling factor: 0.25
240
Scaling factor: 0.25
241
Scaling factor: 0.25
242
Scaling factor: 0.3076923076923077
243
Scaling factor: 0.3076923076923077
244
Scaling factor: 0.3076923076923077
245
Scaling factor: 0.3076923076923077
246
Scaling factor: 0.25
247
Scaling factor: 0.25
248
Scaling factor: 0.25
249
Scaling factor: 0.25
250
Scaling factor: 0.3076923076923077
251
Scaling factor: 0.3076923076923077
252
Scaling factor: 0.25
253
Scaling factor: 0.25
254
Scaling factor: 0.3076923076923077
255
Scaling factor: 0.3076923076923077
256
Scaling factor: 0.3076923076923077
257
Scaling factor: 0.3076923076923077
258
Scaling factor: 0.25
259
Scaling factor: 0.25
260
Scaling factor: 0.25
261
Scaling factor: 0.25
262
Scaling factor: 0.25
263
Scaling factor: 0.25
264
Scaling factor: 0.3076923076923077
265
Scaling factor: 0.3076923076923077
266
Scal

Scaling factor: 0.3076923076923077
491
Scaling factor: 0.3076923076923077
492
Scaling factor: 0.3076923076923077
493
Scaling factor: 0.3076923076923077
494
Scaling factor: 0.3076923076923077
495
Scaling factor: 0.3076923076923077
496
Scaling factor: 0.3076923076923077
497
Scaling factor: 0.3076923076923077
498
Scaling factor: 0.3076923076923077
499
Scaling factor: 0.3076923076923077
500
Scaling factor: 0.3076923076923077
501
Scaling factor: 0.3076923076923077
502
Scaling factor: 0.3076923076923077
503
Scaling factor: 0.3076923076923077
504
Scaling factor: 0.3076923076923077
505
Scaling factor: 0.25
506
Scaling factor: 0.3076923076923077
507
Scaling factor: 0.3076923076923077
508
Scaling factor: 0.3076923076923077
509
Scaling factor: 0.3076923076923077
510
Scaling factor: 0.3076923076923077
511
Scaling factor: 0.3076923076923077
512
Scaling factor: 0.3076923076923077
513
Scaling factor: 0.3076923076923077
514
Scaling factor: 0.3076923076923077
515
Scaling factor: 0.25
516
Scaling factor

...

6558
Scaling factor: 0.2507345739471107
6559
Scaling factor: 0.2507345739471107
6560
Scaling factor: 0.2507345739471107
6561
Scaling factor: 0.2507345739471107
6562
Scaling factor: 0.25
6563
Scaling factor: 0.25
6564
Scaling factor: 0.3076923076923077
6565
Scaling factor: 0.2507345739471107
6566
Scaling factor: 0.2507345739471107
6567
Scaling factor: 0.25
6568
Scaling factor: 0.25
6569
Scaling factor: 0.25
6570
Scaling factor: 0.25
6571
Scaling factor: 0.3076923076923077
6572
Scaling factor: 0.3076923076923077
6573
Scaling factor: 0.3076923076923077
6574
Scaling factor: 0.3076923076923077
6575
Scaling factor: 0.25
6576
Scaling factor: 0.25
6577
Scaling factor: 0.2507345739471107
6578
Scaling factor: 0.2507345739471107
6579
Scaling factor: 0.3076923076923077
6580
Scaling factor: 0.3076923076923077
6581
Scaling factor: 0.25
6582
Scaling factor: 0.25
6583
Scaling factor: 0.3076923076923077
6584
Scaling factor: 0.3076923076923077
6585
Scaling factor: 0.3076923076923077
6586
Scaling factor:

Scaling factor: 0.2507345739471107
6797
Scaling factor: 0.2507345739471107
6798
Scaling factor: 0.2507345739471107
6799
Scaling factor: 0.2507345739471107
6800
Scaling factor: 0.25
6801
Scaling factor: 0.25
6802
Scaling factor: 0.25
6803
Scaling factor: 0.25
6804
Scaling factor: 0.25
6805
Scaling factor: 0.25
6806
Scaling factor: 0.3076923076923077
6807
Scaling factor: 0.3076923076923077
6808
Scaling factor: 0.3076923076923077
6809
Scaling factor: 0.3076923076923077
6810
Scaling factor: 0.3076923076923077
6811
Scaling factor: 0.3076923076923077
6812
Scaling factor: 0.25
6813
Scaling factor: 0.25
6814
Scaling factor: 0.25
6815
Scaling factor: 0.25
6816
Scaling factor: 0.33442194644023515
6817
Scaling factor: 0.33442194644023515
6818
Scaling factor: 0.25
6819
Scaling factor: 0.25
6820
Scaling factor: 0.25
6821
Scaling factor: 0.3076923076923077
6822
Scaling factor: 0.3076923076923077
6823
Scaling factor: 0.25
6824
Scaling factor: 0.25
6825
Scaling factor: 0.25
6826
Scaling factor: 0.25
6

Scaling factor: 0.19122315592903827
7035
Scaling factor: 0.19122315592903827
7036
Scaling factor: 0.19122315592903827
7037
Scaling factor: 0.19122315592903827
7038
Scaling factor: 0.19122315592903827
7039
Scaling factor: 0.19122315592903827
7040
Scaling factor: 0.19122315592903827
7041
Scaling factor: 0.3076923076923077
7042
Scaling factor: 0.3076923076923077
7043
Scaling factor: 0.19122315592903827
7044
Scaling factor: 0.19122315592903827
7045
Scaling factor: 0.19122315592903827
7046
Scaling factor: 0.19122315592903827
7047
Scaling factor: 0.19122315592903827
7048
Scaling factor: 0.19122315592903827
7049
Scaling factor: 0.19122315592903827
7050
Scaling factor: 0.19122315592903827
7051
Scaling factor: 0.19122315592903827
7052
Scaling factor: 0.19122315592903827
7053
Scaling factor: 0.19122315592903827
7054
Scaling factor: 0.19122315592903827
7055
Scaling factor: 0.19122315592903827
7056
Scaling factor: 0.19122315592903827
7057
Scaling factor: 0.19122315592903827
7058
Scaling factor: 0.

7274
Scaling factor: 0.25
7275
Scaling factor: 0.25
7276
Scaling factor: 0.25
7277
Scaling factor: 0.25
7278
Scaling factor: 0.25
7279
Scaling factor: 0.25
7280
Scaling factor: 0.25
7281
Scaling factor: 0.3076923076923077
7282
Scaling factor: 0.3076923076923077
7283
Scaling factor: 0.25
7284
Scaling factor: 0.25
7285
Scaling factor: 0.3076923076923077
7286
Scaling factor: 0.3076923076923077
7287
Scaling factor: 0.25
7288
Scaling factor: 0.25
7289
Scaling factor: 0.3076923076923077
7290
Scaling factor: 0.3076923076923077
7291
Scaling factor: 0.3076923076923077
7292
Scaling factor: 0.3076923076923077
7293
Scaling factor: 0.3076923076923077
7294
Scaling factor: 0.3076923076923077
7295
Scaling factor: 0.25
7296
Scaling factor: 0.25
7297
Scaling factor: 0.3076923076923077
7298
Scaling factor: 0.3076923076923077
7299
Scaling factor: 0.25
7300
Scaling factor: 0.25
7301
Scaling factor: 0.25
7302
Scaling factor: 0.25
7303
Scaling factor: 0.25
7304
Scaling factor: 0.25
7305
Scaling factor: 0.25


Scaling factor: 0.25
7536
Scaling factor: 0.25
7537
Scaling factor: 0.25
7538
Scaling factor: 0.25
7539
Scaling factor: 0.25
7540
Scaling factor: 0.25
7541
Scaling factor: 0.25
7542
Scaling factor: 0.25
7543
Scaling factor: 0.25
7544
Scaling factor: 0.25
7545
Scaling factor: 0.3076923076923077
7546
Scaling factor: 0.3076923076923077
7547
Scaling factor: 0.3076923076923077
7548
Scaling factor: 0.3076923076923077
7549
Scaling factor: 0.25
7550
Scaling factor: 0.25
7551
Scaling factor: 0.25
7552
Scaling factor: 0.25
7553
Scaling factor: 0.3076923076923077
7554
Scaling factor: 0.3076923076923077
7555
Scaling factor: 0.3076923076923077
7556
Scaling factor: 0.3076923076923077
7557
Scaling factor: 0.25
7558
Scaling factor: 0.25
7559
Scaling factor: 0.3076923076923077
7560
Scaling factor: 0.3076923076923077
7561
Scaling factor: 0.25
7562
Scaling factor: 0.25
7563
Scaling factor: 0.25
7564
Scaling factor: 0.25
7565
Scaling factor: 0.3076923076923077
7566
Scaling factor: 0.3076923076923077
7567
